In [1]:
import os, json, sys
import shutil
import threading


import numpy as np
from datetime import datetime
import time

from astropy.io import fits
from astropy.wcs import WCS
from xml.etree import ElementTree as ET

sys.path.append(os.path.abspath('..'))


from utils.data_gen import *
from utils.job_logger import update_job_status, load_job_log
from fileio.process_xml import get_model_info
from fileio.process_xml import parse_job_parameters
from core.model_manager import ModelManager
from utils.aux_fct import tile_filter, first_NMS


In [ ]:
import os, sys


cat_dir = "/home/gsainton/01_CODES/CIANNA_OTF/CODE/client/results"
cat_name = "net0_rts_51046891-3826-416c-8a2a-573a8b559982.dat"
cat_full_path = os.path.join(cat_dir, cat_name)

output_xml_path = os.path.join(cat_dir, "test_pred2xml_output.xml")

print(f"📂 Fichier d'entrée : {cat_full_path}")
print(f"📝 Fichier XML de sortie : {output_xml_path}")

📂 Fichier d'entrée : /home/gsainton/01_CODES/CIANNA_OTF/CODE/client/results/net0_rts_51046891-3826-416c-8a2a-573a8b559982.dat
📝 Fichier XML de sortie : /home/gsainton/01_CODES/CIANNA_OTF/CODE/client/results/test_pred2xml_output.xml


In [ ]:
######	  GLOBAL VARIABLES AND DATA	  #####
map_pixel_size = 32768 # Full SDC1 image size
beam_size = 0.625 #in arcsec
pixel_size = 0.000167847 # In degree

#####    NETWORK RELATED GLOBAL VARIABLES     #####
image_size 	= 256
nb_param  	= 5
nb_box 		= 9
nb_class	= 0
max_epoch 	= 5000
max_nb_obj_per_image = int(340*((image_size*image_size)/(256*256)))


#####    TRAINING RELATED GLOBAL VARIABLES    #####
nb_images_iter 	= 1600
add_noise_prop 	= 0.05 #Proportion of "noise" field examples in nb_images
nb_valid 		= int(100*((256*256)/(image_size*image_size)))

flip_hor 	= 0.5  #total proportion
flip_vert 	= 0.5
rotate_flag = 1

box_clip 	= [5.0,64.0]
flux_clip 	= [1.9e-06, 0.002]
bmaj_clip 	= [0.9, 60.0]
bmin_clip 	= [0.3, 30.0]
pa_res_lim 	= 1.8


#####   INFERENCE RELATED GLOBAL VARIABLES    #####
fwd_image_size = 512
c_size = 16 #Grid element size / reduction factor
yolo_nb_reg = int(fwd_image_size/c_size) #Number of grid element per dimension

overlap = c_size*2
patch_shift = 240
orig_offset = 128

print (orig_offset)

nb_area_w = int((map_pixel_size+2*orig_offset)/patch_shift)
nb_area_h = int((map_pixel_size+2*orig_offset)/patch_shift)

print (nb_area_w, nb_area_h)

nb_images_all = nb_area_w*nb_area_h

val_med_lims = np.array([0.6,0.3,0.1])
val_med_obj  = np.array([0.9,0.7,0.5])

first_nms_thresholds = np.array([0.05,-0.1,-0.3,-0.5]) #lower is stricter
first_nms_obj_thresholds = np.array([1.0,0.7,0.50,0.30])
second_nms_threshold = -0.15


print("Loading prediction data ... ")

pred_data = np.fromfile(cat_full_path, dtype="float32")
print(np.shape(pred_data))
#Repeat corresponds to the number of MC_MODEL realization
repeat = 1
predict = np.reshape(pred_data, (repeat, nb_area_h, nb_area_w, 
                                 nb_box*(8+nb_param),yolo_nb_reg,yolo_nb_reg))
#Only keep the mean, but any realization statistic can be computed here
predict = np.mean(predict, axis=0)

print (np.shape(predict))
print ("1st order predictions filtering ... ")

128
137 137
Loading prediction data ... 
(3249,)


ValueError: cannot reshape array of size 3249 into shape (1,137,137,108,32,32)

In [ ]:

final_boxes = []
c_tile = np.zeros((yolo_nb_reg*yolo_nb_reg*nb_box,(6+1+nb_param+1)),dtype="float32")
c_tile_kept = np.zeros((yolo_nb_reg*yolo_nb_reg*nb_box,(6+1+nb_param+1)),dtype="float32")
c_box = np.zeros((6+1+nb_param+1),dtype="float32")
patch = np.zeros((fwd_image_size, fwd_image_size), dtype="float32")

box_count_per_reg_hist = np.zeros((nb_box+1), dtype="int")

cumul_filter_box = 0; cumul_post_nms = 0

for ph in tqdm(range(0,nb_area_h)):
	for pw in range(0, nb_area_w):

		c_tile[:,:] = 0.0; c_tile_kept[:,:] = 0.0

		p_x = pw; p_y = ph

		xmin = p_x*patch_shift - orig_offset
		xmax = p_x*patch_shift + fwd_image_size - orig_offset
		ymin = p_y*patch_shift - orig_offset
		ymax = p_y*patch_shift + fwd_image_size - orig_offset

		px_min = 0; px_max = fwd_image_size
		py_min = 0; py_max = fwd_image_size

		if(ph == 0 or ph == nb_area_h-1 or pw == 0 or pw == nb_area_w-1):
			patch[:,:] = 0.0
		else:
			patch[:,:] = full_data_norm[ymin:ymax,xmin:xmax]

		c_pred = predict[ph,pw,:,:,:]
		c_nb_box = tile_filter(c_pred, c_box, c_tile, nb_box, prob_obj_cases, patch, val_med_lims, val_med_obj, box_count_per_reg_hist)

		cumul_filter_box += c_nb_box
		c_nb_box_final = c_nb_box
		c_nb_box_final = first_NMS(c_tile, c_tile_kept, c_box, c_nb_box, first_nms_thresholds, first_nms_obj_thresholds)
		cumul_post_nms += c_nb_box_final

		#Manually set all edge predictions to 0
		#The network was not train to interpret these cases which results in artifacts
		out_range = 2
		if(ph < out_range or ph >= nb_area_h-out_range or pw < out_range or pw >= nb_area_w-out_range):
			c_nb_box_final = 0

		final_boxes.append(np.copy(c_tile_kept[0:c_nb_box_final]))

final_boxes = np.reshape(np.array(final_boxes, dtype="object"), (nb_area_h, nb_area_w))

flat_kept = np.vstack(final_boxes.flatten())

print ("NMS removed average frac:", (cumul_filter_box-cumul_post_nms)/cumul_filter_box)


In [8]:
try:
    pred2xml(
        dat_path=cat_full_path,
        output_xml_path=output_xml_path,
        full_data_norm=full_data_norm,
        nb_area_w=nb_area_w,
        nb_area_h=nb_area_h,
        patch_shift=patch_shift,
        orig_offset=orig_offset,
        fwd_image_size=fwd_image_size,
        prob_obj_cases=prob_obj_cases,
        val_med_lims=val_med_lims,
        val_med_obj=val_med_obj,
        first_nms_thresholds=first_nms_thresholds,
        first_nms_obj_thresholds=first_nms_obj_thresholds,
        nb_box=nb_box,
        nb_param=nb_param,
        yolo_nb_reg=yolo_nb_reg
    )
    print("✅ Conversion réussie !")
except Exception as e:
    print(f"❌ Erreur : {e}")

❌ Erreur : cannot reshape array of size 3249 into shape (1,3,3,104,16,16)


In [9]:
from xml.dom import minidom

with open(output_xml_path, "r") as f:
    xml_data = f.read()

pretty_xml = minidom.parseString(xml_data).toprettyxml(indent="  ")
print(pretty_xml[:2000])  # tronqué pour affichage

FileNotFoundError: [Errno 2] No such file or directory: '/home/gsainton/01_CODES/CIANNA_OTF/CODE/client/results/test_pred2xml_output.xml'